In [1]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv

from arcgis.gis import GIS
from arcgis.geocoding import geocode

In [2]:
BASE_DIR = Path(".").absolute().parent
print(BASE_DIR)

/home/ljin/Projects/address_processing


In [3]:
PROJECTS_DIR = BASE_DIR.parent
OCR_DIR = PROJECTS_DIR / "deepsales_business_card_ocr"
EXCELS = OCR_DIR / "excels"

In [4]:
load_dotenv(BASE_DIR/".env")

True

In [5]:
PORTAL = os.environ.get("PORTAL_URL")
PORTAL_ID = os.environ.get("PORTAL_ID")
PORTAL_PW = os.environ.get("PORTAL_PW")
print(PORTAL)

https://portal.esrikr.com/portal


In [6]:
gis = GIS(PORTAL, PORTAL_ID, PORTAL_PW)
print(gis)

GIS @ https://portal.esrikr.com/portal version:10.3


In [62]:
EXCEL_FILE_PATH = EXCELS / "240527_KBGoodJob.xlsx"

In [63]:
df = pd.read_excel(str(EXCEL_FILE_PATH))
df = df.replace({"Nan": " ", "None": " ","NaN": " ", "nan": " "})

In [64]:
def parse_address(address):
    if pd.isna(address):
        return {
            "full_address": "",
            "country": "",
            "state": "",
            "city": "",
            "address1": "",
            "address2": "",
            "x":"",
            "y": "",
            "accuracy": 0,
            "zipcode": ""
        }        
    if address.strip() == "" or address.lower().strip() == "nan":
        return {
            "full_address": "",
            "country": "",
            "state": "",
            "city": "",
            "address1": "",
            "address2": "",
            "x":"",
            "y": "",
            "accuracy": 0,
            "zipcode": ""
        }
    try:
        res = geocode(address, lang_code="ENG")
        for elem in res:
            if elem.get("attributes").get("LangCode") != "ENG":
                continue
            else:
                full_address = elem.get("attributes").get("LongLabel","")
                country = elem.get("attributes").get("CntryName","")
                state = elem.get("attributes").get("Region","")
                city = elem.get("attributes").get("City","")
                address1 = elem.get("attributes").get("StAddr","")
                address2 = elem.get("attributes").get("BldgName","")
                x = elem.get("location").get("x","")
                y = elem.get("location").get("y","")
                accuracy = elem.get("attributes").get("Score",0)
                zipcode = elem.get("attributes").get("Postal","")
                return {
                    "full_address": full_address,
                    "country": country,
                    "state": state,
                    "city": city,
                    "address1": address1,
                    "address2": address2,
                    "x": x,
                    "y": y,
                    "accuracy": accuracy,
                    "zipcode": zipcode
                }
        return {
            "full_address": " ",
            "country": " ",
            "state": " ",
            "city": " ",
            "address1": " ",
            "address2": " ",
            "x": "",
            "y": "",
            "accuracy": 0,
            "zipcode": " "
        }
    except IndexError:
        print(f"{address} cannot be parsed.")
        return {
            "full_address": "",
            "country": "",
            "state": "",
            "city": "",
            "address1": "",
            "address2": "",
            "x":"",
            "y": "",
            "accuracy": 0,
            "zipcode": ""
        }

In [65]:
def get_best_address(row):
    parsed_eng = parse_address(row['address_eng']) if pd.notnull(row['address_eng']) else None
    parsed_nat = parse_address(row['address_nat']) if pd.notnull(row['address_nat']) else None

    if parsed_eng and parsed_nat:
        return parsed_eng if parsed_eng['accuracy'] >= parsed_nat['accuracy'] else parsed_nat
    elif parsed_eng:
        return parsed_eng
    elif parsed_nat:
        return parsed_nat
    else:
        return {
            "full_address": "",
            "country": "",
            "state": "",
            "city": "",
            "address1": "",
            "address2": "",
            "x": "",
            "y": "",
            "accuracy": 0,
            "zipcode": ""
        }

In [66]:
parsed_addresses = df["full_address"].apply(parse_address)

In [67]:
parsed_addresses = pd.DataFrame(parsed_addresses.tolist())

In [68]:
df.columns

Index(['company_name_eng', 'company_name_nat', 'email', 'fax',
       'department_eng', 'department_nat', 'url', 'landline', 'name_eng',
       'name_nat', 'position_eng', 'position_nat', 'telephone', 'country_code',
       'full_address', 'country', 'state', 'city', 'address1', 'address2', 'x',
       'y', 'zipcode'],
      dtype='object')

In [69]:
df = df.drop(columns = ["full_address", "country", "state", "city", "address1", "address2", "x", "y", "zipcode"])
df = pd.concat([df, parsed_addresses], axis=1)
df

,company_name_eng,company_name_nat,email,fax,department_eng,department_nat,url,landline,name_eng,name_nat,...,full_address,country,state,city,address1,address2,x,y,accuracy,zipcode
0,T-Assi,NaN,NaN,NaN,NaN,NaN,www.T-ASSI.co.kr,NaN,NaN,NaN,...,,,,,,,,,0.00,
1,"Gbike Co., Ltd.",(주)지바이크,jay.kim@gbike.io,NaN,Hr & Ga,인사총무팀,NaN,NaN,Jay Kim,김정민,...,"Gangnam-gu, Seoul, KOR",South Korea,Seoul,,,,127.06278,37.4951,76.05,
2,Genoray,제노레이,ghjeon@genoray.com,3.151786e+09,Personnel & General Affairs Team,경영지원실 인사총무팀,www.genoray.com,3.151786e+09,Geonho Jeon,전건호,...,"3-15, Dunchon-daero 80beon-gil, Seongnam-dong,...",South Korea,Gyeonggi-do,Seongnam-si,"3-15, Dunchon-daero 80beon-gil",,127.128451,37.42876,100.00,13383
3,"Dolbomdream Co., Ltd..",돌봄드림,klaus@dolbomdream.com,NaN,NaN,NaN,www.dolbomdream.com,1.066925e+09,Gyuhwan (Klaus) Lee,이규환,...,"4, Daehak-ro 155beon-gil, Gung-dong, Yuseong-g...",South Korea,Daejeon,Yuseong-gu,"4, Daehak-ro 155beon-gil",,127.350777,36.360985,100.00,34138
4,Concentrix,콘센트릭스 서비스 코리아,TA@concentrix.com,NaN,NaN,NaN,www.concentrix.com,NaN,NaN,NaN,...,"662, Gyeongin-ro, Sindorim-dong, Guro-gu, Seou...",South Korea,Seoul,Guro-gu,"662, Gyeongin-ro",,126.889278,37.508821,100.00,08209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Lookoptics Co., Ltd",(주)룩옵틱스,khseo@lookoptics.co.kr,2.301683e+08,Hr,인재개발팀,www.lookoptics.co.kr,2.301682e+08,Kwang Hwi Seo,서광휘,...,"19, Seoun-ro, Seocho-dong, Seocho-gu, Seoul, KOR",South Korea,Seoul,Seocho-gu,"19, Seoun-ro",,127.029063,37.485592,100.00,06732
69,"Nds Solutions Co., Ltd.",NaN,chloe.park@netds.net,3.128132e+08,Overseas Sales Dept.,해외영업부,www.netds.net,3.128134e+08,Jungeun (Chloe) Park,박정은,...,"120, Heungdeokjungang-ro, Yeongdeok-dong, Gihe...",South Korea,Gyeonggi-do,Yongin-si,"120, Heungdeokjungang-ro",,127.080097,37.27489,100.00,16950
70,"Goodmorning Information Technology Co., Ltd.",굿모닝아이텍(주),gipark@goodmit.co.kr,5.053752e+09,Management Support Division,경영지원본부/인사총무팀,www.goodmit.co.kr,2.377523e+08,Park Gwang Ik,박광익,...,"434, World Cup buk-ro, Sangam-dong, Mapo-gu, S...",South Korea,Seoul,Mapo-gu,"434, World Cup buk-ro",,126.88605,37.581622,100.00,03922
71,Shinwoo Valve,신우밸브주식회사,jhpark2@shinwoovalve.com,3.198324e+08,Human Resources,총무부,http://www.swvis.com,3.198094e+08,Junghwan Park,박정환,...,,,,,,,,,0.00,


In [55]:
df.columns

Index(['company_name_eng', 'company_name_nat', 'email', 'fax',
       'department_eng', 'department_nat', 'url', 'landline', 'name_eng',
       'name_nat', 'position_eng', 'position_nat', 'telephone', 'country_code',
       'full_address', 'country', 'state', 'city', 'address1', 'address2', 'x',
       'y', 'accuracy', 'zipcode'],
      dtype='object')

In [14]:
parsed_addresses = df.apply(get_best_address, axis=1)

In [15]:
parsed_df = pd.DataFrame(parsed_addresses.tolist())
parsed_df

,full_address,country,state,city,address1,address2,x,y,accuracy,zipcode
0,"1105, Baekhyeon-ri, Sandong-eup, Gumi-si, Gyeo...",South Korea,Gyeongsangbuk-do,Gumi-si,1105,,128.455133,36.192999,97.78,
1,"255-1, Changsin-dong, Jongno-gu, Seoul, KOR",South Korea,Seoul,Jongno-gu,255-1,,127.015207,37.573814,100.00,
2,"401, Hakdong-ro, Cheongdam-dong, Gangnam-gu, S...",South Korea,Seoul,Gangnam-gu,"401, Hakdong-ro",,127.041673,37.517641,97.30,06069
3,"37, Seongseogongdannam-ro, Woram-dong, Dalseo-...",South Korea,Daegu,Dalseo-gu,"37, Seongseogongdannam-ro",,128.49639,35.830766,97.65,42718
4,"Bucheon-si, Gyeonggi-do, KOR",South Korea,Gyeonggi-do,Bucheon-si,,,126.78306,37.49889,74.25,
...,...,...,...,...,...,...,...,...,...,...
374,"7-3, Jeongdeun-ro 6-gil, Ido 2(i)-dong, Jeju-s...",South Korea,Jeju-do,Jeju-si,"7-3, Jeongdeun-ro 6-gil",,126.542954,33.498831,100.00,63225
375,"Seodaemun-gu, Seoul, KOR",South Korea,Seoul,,,,126.93506,37.57809,74.20,
376,"35, Gwangnaru-ro 6-gil, Seongsu-dong2(i)-ga, S...",South Korea,Seoul,Seongdong-gu,"35, Gwangnaru-ro 6-gil",,127.063297,37.5468,97.17,04799
377,"628-7, Deungchon-dong, Gangseo-gu, Seoul, KOR",South Korea,Seoul,Gangseo-gu,628-7,,126.855269,37.559469,94.22,


In [16]:
df = pd.concat([df, parsed_df], axis=1)

In [70]:
df.columns

Index(['company_name_eng', 'company_name_nat', 'email', 'fax',
       'department_eng', 'department_nat', 'url', 'landline', 'name_eng',
       'name_nat', 'position_eng', 'position_nat', 'telephone', 'country_code',
       'full_address', 'country', 'state', 'city', 'address1', 'address2', 'x',
       'y', 'accuracy', 'zipcode'],
      dtype='object')

In [19]:
df = df.drop(columns=["address_eng","address_nat","accuracy"])

In [71]:
df["country"].unique()

array(['', 'South Korea'], dtype=object)

In [57]:
COUNTRY_CODE_DICT = {
    "South Korea": "82",
    "China": "86",
    "Türkiye": "90"
}

In [72]:
def match_country_code(country, country_code_dict=COUNTRY_CODE_DICT):
    if pd.isna(country):
        return ""
    elif country.strip() == "":
        return ""
    return country_code_dict.get(country, "")

In [73]:
df["country_code"] = df["country"].apply(match_country_code)

In [74]:
df.to_excel(str(EXCEL_FILE_PATH), index=False)

In [75]:
df["company_name_eng"].unique()

array(['T-Assi', 'Gbike Co., Ltd.', 'Genoray', 'Dolbomdream Co., Ltd..',
       'Concentrix', 'Gmb Korea Corp.', 'Shinhancard', 'Unisem Co., Ltd.',
       'Kjc Display Corporation', 'Vivastudio Corp.', 'Samwon Polytech',
       'Seongnam Campus Of Korea Polytechnic', 'Nhr', 'Robocare', 'Ehwa',
       'L&C Bio Co., Ltd.', 'Pemtron Corporation',
       'Korea Arlico Pharm. Co., Ltd.', 'Serim B&G Co.,Ltd',
       'Fine Semitech Corp.', 'Ray Co., Ltd.', 'Evsis', 'Mpc Plus',
       'Widenet Engineering', 'H1 Construction Co., Ltd.',
       'Hilight Brands', 'Luxco', 'Bcworld Pharm ', 'Clobot',
       'Soon Chun Hyang University', 'V-Space', 'Teso Engineering',
       'Woowon Development', 'Teamgrit', 'Hecto Financial',
       'Mftech Co., Ltd.', 'Chabot Mobility', 'Ap Systems',
       'Atecc& Co., Ltd.', 'Kwangjang Architects & Engineers', 'Tripbtoz',
       'Lomin', 'Stats Chippac Korea Ltd.', 'Navifra', 'Lt Metal',
       'Korean Master Hand Myung Jang 10 Bakery Café',
       'Opas Net. C